<a href="https://colab.research.google.com/github/EddyGiusepe/Recommendation_of_MyAnimeList_2021/blob/main/RecomendacaoAnime_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h2 align='center'>Aplicando Regressão a Dados de Anime2020</h2> 


**Cientista de Dados Jr.:**  Dr.Eddy Giusepe Chirinos Isidro

Este Script está baseado no [Kaggle - Recomendação Animes2020](https://github.com/Alejandro1848/Curso-Machine-Learning-YouTube/blob/main/Regresi%C3%B3n_Ridge_Ejemplo2.ipynb).

In [1]:
# Importamos nossas bibliotecas

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
# Carregamos nossos Dados

df=pd.read_csv('/content/drive/MyDrive/1_Kaggle_TensorFlow_linksJORGE_etc/AnimesList_2020/anime.csv')


In [4]:
# Visualizamos nossos Dados

df.sample(5)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
11704,33087,Meiji Tokyo Renka Movie 2: Hanakagami no Fantasia,6.32,"Fantasy, Harem, Historical, Romance, Shoujo",Unknown,劇場版 明治東亰恋伽 〜花鏡の幻想曲〜,Movie,1,"May 6, 2016",Unknown,MAGES.,Unknown,Studio Deen,Game,59 min.,PG-13 - Teens 13 or older,6789.0,7758,3101,11,169,366,99,94,2373,33.0,15.0,28.0,64.0,42.0,26.0,6.0,6.0,3.0,11.0
15014,38618,Shimajirou to Ururu no Hero Land,Unknown,Kids,Unknown,しまじろうとうるるのヒーローランド,Movie,1,"Mar 15, 2019",Unknown,Benesse Corporation,Unknown,The Answer Studio,Original,Unknown,G - All Ages,11940.0,16253,125,0,1,23,1,24,76,1.0,Unknown,Unknown,1.0,4.0,4.0,1.0,1.0,1.0,4.0
12285,34182,Nikoniko♪Connie-chan,Unknown,Kids,Unknown,ニコニコ♪コニーちゃん,TV,5,"Apr, 2009",Spring 2009,Unknown,Unknown,Unknown,Other,10 min. per ep.,G - All Ages,11168.0,16632,108,0,10,29,1,38,30,3.0,1.0,4.0,3.0,2.0,2.0,1.0,Unknown,Unknown,3.0
16113,40361,"Shiro no Kanga""roo"" no Uta",Unknown,"Music, Kids","White kanga""roo""'s song","しろのカンガ""ルー""のうた",ONA,1,"Sep 1, 2019",Unknown,Unknown,Unknown,Unknown,Original,48 sec.,G - All Ages,12030.0,15603,160,0,4,115,4,13,24,2.0,Unknown,Unknown,Unknown,6.0,19.0,11.0,21.0,13.0,14.0
8573,21971,Sore Ike! Anpanman: Yuuki no Hana ga Hiraku Toki,Unknown,"Comedy, Fantasy, Kids",Unknown,それいけ!アンパンマン 勇気の花がひらくとき,Movie,1,"Jul 24, 1999",Unknown,Tokyo Movie Shinsha,Unknown,Unknown,Unknown,56 min.,PG - Children,12201.0,14768,206,0,9,83,3,54,57,8.0,3.0,2.0,5.0,7.0,9.0,3.0,3.0,1.0,3.0


In [5]:
df.shape

(17562, 35)

In [6]:
pd.set_option('display.max_columns',35)

In [13]:
df.sample(5)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
3155,3588,Soul Eater,7.88,"Action, Fantasy, Comedy, Supernatural, Shounen",Soul Eater,ソウルイーター,TV,51,"Apr 7, 2008 to Mar 30, 2009",Spring 2008,"TV Tokyo, Aniplex, Dentsu, Media Factory",Funimation,Bones,Manga,24 min. per ep.,PG-13 - Teens 13 or older,700.0,43,1218423,23856,87632,763245,77117,65962,224467,76241.0,134353.0,201025.0,152877.0,57717.0,24376.0,8704.0,2853.0,1404.0,1351.0
10326,30074,Symphonic Variations,5.24,"Music, Kids",Unknown,シンフォニック・バリエーション,Music,1,"Jun 7, 1976",Unknown,NHK,Unknown,Unknown,Original,2 min.,G - All Ages,10253.0,13749,278,0,5,203,1,29,40,11.0,1.0,5.0,11.0,19.0,34.0,20.0,18.0,11.0,14.0
16439,40924,22/7: 8＋3=?,6.54,Music,Unknown,22/7:8＋3=?,Special,1,"Sep 16, 2020",Unknown,Aniplex,Unknown,A-1 Pictures,Other,24 min.,PG-13 - Teens 13 or older,5658.0,6698,4657,8,71,2410,25,42,2109,102.0,76.0,224.0,555.0,474.0,213.0,80.0,37.0,16.0,13.0
8802,22973,Sangokushi,Unknown,"Action, Historical",Unknown,三国志,Special,1,"Jan 4, 1982",Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,PG-13 - Teens 13 or older,11796.0,14186,247,0,16,69,6,44,112,4.0,6.0,4.0,7.0,5.0,7.0,3.0,Unknown,1.0,4.0
10540,30430,Kimi wa Sou Kimeta,5.24,Music,You Just Decided,君はそう決めた,Music,1,"Nov 15, 2011",Unknown,Unknown,Unknown,Unknown,Unknown,5 min.,G - All Ages,10263.0,14291,238,0,5,175,1,18,39,6.0,Unknown,3.0,14.0,12.0,30.0,17.0,18.0,9.0,15.0


In [14]:
# Substitui "unknow" --> 0

df = df.replace('Unknown', float(0)) 

In [16]:
df.sample(2)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
3426,3989,Suna no Bara: Yuki no Mokushiroku,6.27,Action,0,砂の薔薇[デザート・ローズ] 雪の黙示録,OVA,1,"Apr 25, 1993",0,"Toei Video, Hakusensha",0,J.C.Staff,Manga,44 min.,R+ - Mild Nudity,7016.0,8765,2043,4,83,893,31,35,1001,19.0,18.0,77.0,179.0,196.0,119.0,37.0,17.0,4.0,2.0
581,618,Juubee Ninpuuchou: Ryuuhougyoku-hen,6.7,"Adventure, Horror, Magic, Martial Arts, Samura...",Ninja Scroll:The Series,獣兵衛忍風帖「龍宝玉篇」,TV,13,"Apr 15, 2003 to Jul 15, 2003",Spring 2003,Studio Take Off,"Discotek Media, Urban Vision",Madhouse,Original,24 min. per ep.,R - 17+ (violence & profanity),4899.0,2798,33722,89,1178,18870,1006,1543,11125,742.0,1340.0,2792.0,4165.0,2856.0,1721.0,907.0,395.0,255.0,154.0


In [17]:
# Aqui eliminamos as colunas que não são relevantes para a nossa análise

df=df.drop(['MAL_ID', 'Name', 'Genres', 'English name', 'Japanese name', 'Type', 'Aired', 'Premiered', 'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Score-10',
            'Score-9', 'Score-8', 'Score-7', 'Score-6', 'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],axis=1)


In [18]:
df.head()

,Score,Episodes,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch
0,8.78,26,39,1251960,61971,105808,718161,71513,26678,329800
1,8.39,1,518,273145,1174,4143,208333,1935,770,57964
2,8.24,26,201,558913,12944,29113,343492,25465,13925,146918
3,7.27,26,1467,94683,587,4300,46165,5121,5378,33719
4,6.98,52,4369,13224,18,642,7314,766,1108,3394


In [20]:
df=df.astype(float)

In [21]:
df.shape

(17562, 10)

In [26]:
df = df[df.Score > 0.0]

In [27]:
df.head(2)

,Score,Episodes,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch
176,8.83,1.0,41.0,1245727.0,25610.0,14444.0,1124666.0,3124.0,1437.0,102056.0
741,9.07,110.0,633.0,230168.0,13834.0,25968.0,52583.0,14911.0,5950.0,130756.0


In [28]:
df.isnull().sum() 

Score            0
Episodes         0
Popularity       0
Members          0
Favorites        0
Watching         0
Completed        0
On-Hold          0
Dropped          0
Plan to Watch    0
dtype: int64

In [29]:
df.columns

Index(['Score', 'Episodes', 'Popularity', 'Members', 'Favorites', 'Watching',
       'Completed', 'On-Hold', 'Dropped', 'Plan to Watch'],
      dtype='object')